In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model = "qwen3:4b",
    base_url="http://localhost:11434/v1",
    api_key="ollama",
)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001BF20095850>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001BF2009D790>, root_client=<openai.OpenAI object at 0x000001BF1EDE6D80>, root_async_client=<openai.AsyncOpenAI object at 0x000001BF20097170>, model_name='qwen3:4b', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://localhost:11434/v1')

In [5]:
from langchain_core.messages import HumanMessage
response = llm.invoke([
    HumanMessage(content="What's the capital of France?")
])

print(response.content)

<think>
Okay, the user is asking for the capital of France. I remember that France is a country in Europe, and its capital is a major city. The most common answer I've heard is Paris. But wait, I should make sure there's no confusion with other cities. Let me think. France has several important cities like Lyon, Marseille, and Bordeaux, but the capital is definitely Paris. I think Paris is the seat of government and the main administrative center. Also, it's well-known for its historical significance and landmarks like the Eiffel Tower and Louvre Museum. I'm pretty confident that the answer is Paris. But maybe the user is testing if I know that sometimes people might confuse other cities, so I should confirm that it's indeed Paris. Yes, I'm sure about that.
</think>

The capital of France is **Paris**. It is the country's largest city, political, cultural, and administrative center, and home to iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.


In [ ]:
from openai import OpenAI, ChatOpenAI
import openai
import json
from typing import List
from pydantic import BaseModel

client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
client

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",
)

In [2]:
response = client.chat.completions.create(
  model="deepseek-r1:8b",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

<think>
Alright, let me tackle this step by step. The user is asking where the 2020 World Series was played.

First, from the conversation history, I see that the user initially asked who won and I confirmed that the LA Dodgers were the winners. Now, they’re following up with a location question.

I know that in 2020, COVID-19 was a big deal, so probably it wasn't held at the usual venues. Arlington-Fort Worth was hosting the World Series that year because of health protocols. Texas had fewer restrictions, making it a safer option for players and staff.

The Dodgers won in six games against the Tampa Bay Rays. I should mention both the location, the teams, and maybe add a bit about the impact of COVID-19 rules to give context.

Let me make sure to structure this clearly. Start with the location, then who played, how many games it took, and maybe a brief note on why the venue was chosen.
</think>

The 2020 World Series was played at Globe Life Field in Arlington, Texas. The Los Angeles 

In [32]:
def structured_output_chatbot(question:str):
    command = f'''
    YOU MUST STRICTLY ADHERE TO THE GIVEN FORMAT.
    format your response so that is follows a JSON format, for example:
    //
    {{
    "location": "Globe Life Field", "city": "Arlington"
    }}

    your response should only contain the JSON format and nothing else.
    '''

    response = client.chat.completions.create(
    model="llama3.2:latest",
    response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"{question}"},
        {"role": "user", "content": f"{command}"}
    ]
    )

    result = json.loads(response.choices[0].message.content)
    return result

question = "Where was the world series in 2020 played?"
print(structured_output_chatbot(question))

{'location': 'Hard Rock Stadium', 'city': 'Miami Gardens'}


In [ ]:
from openai import OpenAI
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

completion = client.beta.chat.completions.parse(
    model="qwen3:4b",
    messages=[
        {
            "role": "system",
            "content": "You are the Smart Scheduling AgetnExtract the names and ages of the people mentioned in the following text."
        },
        {
            "role": "user",
            "content": "John is 30 years old and his sister Alice is 25."
        }
    ],
    tools=[
        openai.pydantic_function_tool(Person)
    ]
)

results = [i.function.parsed_arguments for i in completion.choices[0].message.tool_calls]
results

[Person(name='John', age=30), Person(name='Alice', age=25)]

In [29]:
class Pet(BaseModel):
    name: str
    animal: str
    age: int
    color: str | None
    favorite_toy: str | None

try:
    completion = client.beta.chat.completions.parse(
        temperature=0,
        model="llama3.2:latest",
        messages=[
            {"role": "user", 
            "content": '''
                I have two pets.
                A cat named Luna who is 5 years old and loves playing with yarn. She has grey fur.
                I also have a 2 year old black cat named Loki who loves tennis balls.
            '''}
        ],
        tools=[openai.pydantic_function_tool(Pet)]
    )
    results = [i.function.parsed_arguments for i in completion.choices[0].message.tool_calls]
    print(results)

except Exception as e:
    if type(e) == openai.LengthFinishReasonError:
        print("Too many tokens: ", e)
        pass
    else:
        print(e)
        pass

[Pet(name='Luna', animal='cat', age=5, color='grey', favorite_toy='yarn'), Pet(name='black cat', animal='cat', age=2, color='', favorite_toy='tennis balls')]
